In [ ]:
#拷贝数目变异 (CNV/CNP)也称拷贝数目多态，是一种大小介于1kb至3Mb的DNA片段的变异
#人类2万多个基因平均编码区长度在1.7kb左
右，小于1kb的基因有7533个，1kb-2kb的基
因有7566个，大于2kb基因的有5643个。
#外显子的区别大概在200bp左右
#按照CNV是否致病可分为致病性CNV、非致病性CNV.不明临床意义
CNV
vdz基因的重排
所以说CNV是跨基因的突变
但是也有cnv是正常的，他和人类是共存的


In [ ]:
#体细胞拷贝数变异 (:somatic CNV) :在TCGA数据库里面我们通常关心的是somaticCNV
也就是剔除正常对照里面的CNV多态性信息的肿瘤样本拷贝数变异，只有这些somatic CNV才更可能是跟肿瘤相关的。
CNV最重要的是产前诊断，比如：猫叫综合征

In [ ]:
#用inferCNV判断肿瘤细胞的CNV事件通常包括以下几个步
骤:
1) 样本的基础质控和注释;
0选择合适的reference;
依据基因在染色体上的位置对基因进行排序;
6数据处理，包括肿瘤细胞与ref的信号比较去除、数据均一化处理、降低噪音等过程;
5)CNV最终的预测。从分析过程中来讲，inferCNV需要的输入文件包括:表达矩阵、细胞注释信息、基因在染色体上的位置信息。

In [ ]:
#如果只用免疫细胞作为reference的话，很容易出现假阳性，原因：加入免疫细胞表达CD3D，但是上皮细胞不表达CD3D，因此就会显示基因缺失，出现假阳性的情况
#一个基因的表达的情况是由上下各50各基因共同的表达情况。所以一般是101的片段，这一般被称为100个窗口，100-gene sliding window

#refercen
#最好是拿正常上皮做对照，要不然就是拿免疫间质 尽量多的细胞类型作为对照，最大限度保证结果可信

In [ ]:
#InferCNV算法的详细步骤涉及以下内容:
1)过滤基因:从计数矩阵中删除那些在少于“min_cells per gene”中表达的基因，这一步类似于样本质控过程中的基因去除。2)测序深度的归一化(总和归一化):read counts per cell are scaled to sum to the median total read count across ces。值不是每百万计数(cpm)等指标，而是每中位数总和的计数(这一点区别于Seurat分析单细胞的均一化)。3)对数转换:单个矩阵值(x)转换为log(x+1)，这里对数转换的作用与Seurat分析中的相同。4)centerbynormalgeneexpresion:从对应基因的所有细胞中减去正常(参考)细胞中每个基因的平均值。由于此减法是在对数空间中执行的，因此这有效地导致了相对于正常细胞平均值的对数倍变化值。
5)对数倍数变化值的阅值动态范围。abs(log(x+l)超过'max centered threshold(defaul=-3)的任何值都被设定为该值(设置了最高上限)。
6)chromosome-levelsmoothing:对于每个细胞，沿每个染色体排序的基因具有使用加权运行平均值拟合的表达强度。 默认情况下这是一个包含101个基因的窗口，具有pyramidinal weighting scheme。
7)centering cells:如果大多数基因不在 CNV 区域中，每个细胞的中心表达强度中值设定为零。
8)相对于正常细胞的调整:再次从肿瘤细胞中减去正常值的平均值。 这进一步补偿了拟合处理后产生的差异。
9)log转换被还原，这使得amplification或 deletion的证据在平均值周围更加对称。
上述就是推断CNV分析的基本过程，但是通常为了更加准确的推断CNV事件，往往还要添加两个步骤de-noising filters和HIMIMS算法。
#以上归一化、中心化就是去除离散值等方法，包括下面的降低噪音的方法

In [ ]:
#降低噪音有三种方法
# 第一个是硬阈值，就比如设置0.1为过滤阈值，那么0.9！1.1以外的都被判为CNV，这个也是默认过滤方法
# 第二个是动态变化，一般通过设置sd_samplifer来进行调增阈值，官方给的是1.5，也就是1.5*reference基因表达的标准差进行过滤
# 第三个是软阈值
#infercnv支持两种基于HMM的CNV预测模型，i3和i6两种模式
#i3 就是loss normal gain
#i6就是
state 1 :0x = complete loss
state 2 : 0.5x = loss of one copy
state 3 : 1x = neutral
state 4 : 1.5x = addition of one copy
state 5 : 2x = addition of two copies
state 6 : 3x = essentially a placeholder for >2x copies but modeled as 3x.
#cnv进化树 UPhyloplot2  一般就是cnv变异的越多，认为它发生cnv越早，但是恶性程度也会变大

In [ ]:
#下游分析一般就是cnv分布、肿瘤细胞克隆亚型分析（cnv聚类）、共有cnv和基因功能分析